In [6]:
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pylab as plt
from torch.utils.data import Dataset, DataLoader
import copy
from tqdm import tqdm
import torch.optim.lr_scheduler as sched
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_auc_score, roc_curve
import pickle
import os

In [7]:
root = os.getcwd()
data_root = os.path.join(root, 'data')

### Prepare Data

In [8]:
train_X = pickle.load(open(os.path.join(data_root, 'train_X.pickle'), "rb"))
train_y = pickle.load(open(os.path.join(data_root, 'train_y.pickle'), "rb"))
test_X = pickle.load(open(os.path.join(data_root, 'test_X.pickle'), "rb"))
test_y = pickle.load(open(os.path.join(data_root, 'test_y.pickle'), "rb"))

In [17]:
train_X

array([[-1.3314788 , -1.62203616, -0.42291773, ..., -0.20868932,
        -0.06100026, -0.2074773 ],
       [-1.55302366, -0.86954242,  0.35698963, ..., -0.20868932,
        -0.06100026, -0.2074773 ],
       [-0.4818794 , -0.74390202, -1.25057452, ..., -0.20868932,
        -0.06100026, -0.2074773 ],
       ...,
       [-1.39108681, -1.43350944,  1.28014528, ..., -0.20868932,
        -0.06100026, -0.2074773 ],
       [ 1.4018607 , -1.60088012, -0.08867172, ..., -0.20868932,
        -0.06100026,  4.81980448],
       [ 0.14508991,  0.5273906 , -0.21600353, ..., -0.20868932,
        -0.06100026, -0.2074773 ]])

In [19]:
np.array(train_y)

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [ ]:
import torch
from torch.utils.data import Dataset, RandomSampler, SequentialSampler, DataLoader

class TweetDataset(Dataset):
    def __init__(self, list_of_lists_of_tokens,labels):
        self.input_tensors = []
        self.labels = labels
        
        for sample in list_of_lists_of_tokens:
            self.input_tensors.append(torch.tensor([sample[:]], dtype=torch.float))
            
            
    
    def __len__(self):
        return len(self.input_tensors)
    
    def __getitem__(self, idx):
        # return a (input, target) tuple
        #torch.LongTensor(self.sentences[key])
        return (self.input_tensors[idx], self.labels[idx])

In [27]:
labels = np.array(train_y)


In [36]:
train_y

,hospital_death
29631,0
25212,0
54893,0
46432,0
85876,0
...,...
21243,0
45891,0
42613,0
43567,0


In [40]:
len(labels)

73370

In [42]:
idx = 73369
label_0 = labels[idx]
torch.tensor(label_0, dtype=torch.long)

tensor([1])

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class ICUDataset(Dataset):
    
    def __init__(self, input_array, label_array):
        # Args:
        #      input_array: list of list
        #      label_array: list of list
        
        super(ICUDataset, self).__init__()
        self.input_tensors = []
        self.labels = np.array(label_array)
        for sample in input_array:
            self.input_tensors.append(torch.tensor([sample[:]], dtype=torch.float))
        
    def __len__(self):
        return len(self.input_tensors)

    def __getitem__(self, index):
        x = self.input_tensors[index]
        # want to have y as tensor([1]), for 1 being class
        y = torch.tensor(self.labels[index], dtype=torch.long)
                   
        return x, y

In [44]:
data_train = ICUDataset(train_X, train_y)
data_test = ICUDataset(test_X, test_y)

In [45]:
# Test data train
x, y = next(iter(data_train))
x.shape
y.shape

In [49]:
data_train_loader = DataLoader(data_train, batch_size = 64, shuffle=True, num_workers=4)
data_test_loader = DataLoader(data_test, batch_size = 64, shuffle=False, num_workers=4)

In [50]:
# We want to have x.shape = [batch_size, height, width = len of input]
# y.shape = [batch_size, 1] because y is a class
x, y = next(iter(data_train_loader))